In [15]:
!cd ..

In [16]:
!pwd

/Users/xupeng/Desktop/Mac/python-jupyter代码


In [6]:
import os
import numpy as np
import codecs
import pandas as pd
import json
from glob import glob
import cv2
import shutil
from sklearn.model_selection import train_test_split
from IPython import embed
#1.标签路径
csv_file = "mb.csv"
saved_path = "Mb/"                #保存路径
image_save_path = "Mb/JPEGImages/"
image_raw_parh = "Train/"
#2.创建要求文件夹
if not os.path.exists(saved_path + "Annotations"):
    os.makedirs(saved_path + "Annotations")
if not os.path.exists(saved_path + "JPEGImages/"):
    os.makedirs(saved_path + "JPEGImages/")
if not os.path.exists(saved_path + "ImageSets/Main/"):
    os.makedirs(saved_path + "ImageSets/Main/")
    
#3.获取待处理文件
total_csv_annotations = {}
annotations = pd.read_csv(csv_file,header=None,encoding= 'gb2312').values

for annotation in annotations:
    key = annotation[2].split(os.sep)[-1]
    value = np.array([annotation[1:]])
    if key in total_csv_annotations.keys():
        total_csv_annotations[key] = np.concatenate((total_csv_annotations[key],value),axis=0)
    else:
        total_csv_annotations[key] = value

In [7]:
#4.读取标注信息并写入 xml
cd=0
i=1
j=0
for filename,label in total_csv_annotations.items():
    #embed()
    print('filename:'+filename)
    c=[]
    if i==1:
        i=0
        continue
    for cname in label:
        c.append(cname)
    #print(c[0][0])
    filename = c[0][0]+'_S' + c[0][3]+'_C'+c[0][4]+'_P' + c[0][5] + '.jpg'
    print(filename)
    cd=cd+1
    
    #height, width, channels = cv2.imread(image_raw_parh + filename).shape
    #embed()
    with codecs.open(saved_path + "Annotations/"+filename.replace(".jpg",".xml"),"w","utf-8") as xml:
        xml.write('<annotation>\n')
        xml.write('\t<folder>' + 'GJ2020-DET' + '</folder>\n')
        xml.write('\t<filename>' + filename + '</filename>\n')
        xml.write('\t<source>\n')
        xml.write('\t\t<database>GJ202007</database>\n')
        xml.write('\t\t<annotation>GJ2020</annotation>\n')
        #xml.write('\t\t<image>flickr</image>\n')
        #xml.write('\t\t<flickrid>NULL</flickrid>\n')
        xml.write('\t</source>\n')
        #xml.write('\t<owner>\n')
        #xml.write('\t\t<flickrid>NULL</flickrid>\n')
        #xml.write('\t\t<name>'+label[0][2]+'</name>\n')
        #xml.write('\t</owner>\n')
        xml.write('\t<size>\n')
        xml.write('\t\t<width>'+ str(5120) + '</width>\n')
        xml.write('\t\t<height>'+ str(5120) + '</height>\n')
        xml.write('\t\t<depth>' + str(3) + '</depth>\n')
        xml.write('\t</size>\n')
        xml.write('\t\t<segmented>0</segmented>\n')

        #if isinstance(label,float):
        #    ## 空白
        #    xml.write('</annotation>')
        #    continue
        for label_detail in label:
            labels = label_detail
            print(labels)
            #embed()
            x,y,w,h=0,0,0,0
            X = int(labels[7])
            Y = int(labels[8])
            W = int(labels[11])
            H = int(labels[12])
            '''if c[0][4]==2:
                x=Y
                y=5120-X-W
                w=H
                h=W
            elif c[0][4]==1:
                x = 5120 - Y - H
                y = X
                w = H
                h = W'''
            
            xmin = X
            ymin = Y
            xmax = X+W
            ymax = Y+H
            print(X,Y,W,H)
            print(xmin,ymin,xmax,ymax)
            t=10
            if c[0][6]=='+其他+':
                t=0
            elif c[0][6]=='0':
                t=1
            elif c[0][6]=='边裂':
                t=2
            elif c[0][6]=='边损':
                t=3
            elif c[0][6]=='花纹边':
                t=4
            elif c[0][6]=='局部开裂':
                t=5
            elif c[0][6]=='毛刺':
                t=6
            elif c[0][6]=='头叠':
                t=7
            elif c[0][6]=='折叠':
                t=8
            label_ = t
            
            if xmax <= xmin:
                pass
            elif ymax <= ymin:
                pass
            else:
                xml.write('\t<object>\n')
                xml.write('\t\t<name>'+str(label_)+'</name>\n')
                xml.write('\t\t<pose>Unspecified</pose>\n')
                xml.write('\t\t<truncated>1</truncated>\n')
                xml.write('\t\t<difficult>0</difficult>\n')
                xml.write('\t\t<bndbox>\n')
                xml.write('\t\t\t<xmin>' + str(xmin) + '</xmin>\n')
                xml.write('\t\t\t<ymin>' + str(ymin) + '</ymin>\n')
                xml.write('\t\t\t<xmax>' + str(xmax) + '</xmax>\n')
                xml.write('\t\t\t<ymax>' + str(ymax) + '</ymax>\n')
                xml.write('\t\t</bndbox>\n')
                xml.write('\t</object>\n')
                j=j+1
                
                print(str(cd)+'---'+str(j))
        xml.write('</annotation>')
print(j)        
#6.split files for txt
txtsavepath = saved_path + "ImageSets/Main/"
ftrainval = open(txtsavepath+'/trainval.txt', 'w')
ftest = open(txtsavepath+'/test.txt', 'w')
ftrain = open(txtsavepath+'/train.txt', 'w')
fval = open(txtsavepath+'/val.txt', 'w')
total_files = glob(saved_path+"./Annotations/*.xml")
total_files = [i.split("/")[-1].split(".xml")[0] for i in total_files]
#test_filepath = ""
for file in total_files:
    ftrainval.write(file + "\n")

# move images to voc JPEGImages folder
for image in glob(image_raw_parh+"/*.jpg"):
    shutil.copy(image,saved_path+image_save_path)

train_files,val_files = train_test_split(total_files,test_size=0.15,random_state=42)

for file in train_files:
    ftrain.write(file + "\n")
#val
for file in val_files:
    fval.write(file + "\n")

ftrainval.close()
ftrain.close()
fval.close()
#ftest.close()
print(cd)

filename:newColumn
filename:93802608121
93802608_S1_C2_P1.jpg
['93802608' '93802608121' '1' '1' '2' '1' '局部开裂' '2618' '1980' '0' '0'
 '627' '426' '0' '0' '1']
2618 1980 627 426
2618 1980 3245 2406
1---1
filename:93814448112
93814448_S1_C1_P2.jpg
['93814448' '93814448112' '0' '1' '1' '2' '局部开裂' '1654' '2244' '0' '0'
 '105' '87' '0' '0' '1']
1654 2244 105 87
1654 2244 1759 2331
2---2
filename:93814450221
93814450_S2_C2_P1.jpg
['93814450' '93814450221' '0' '2' '2' '1' '局部开裂' '4735' '1906' '0' '0'
 '91' '102' '0' '0' '1']
4735 1906 91 102
4735 1906 4826 2008
3---3
filename:93814449221
93814449_S2_C2_P1.jpg
['93814449' '93814449221' '0' '2' '2' '1' '局部开裂' '1393' '4312' '0' '0'
 '1225' '264' '0' '0' '1']
1393 4312 1225 264
1393 4312 2618 4576
4---4
filename:93814464211
93814464_S2_C1_P1.jpg
['93814464' '93814464211' '0' '2' '1' '1' '局部开裂' '1946' '1074' '0' '0'
 '75' '400' '0' '0' '1']
1946 1074 75 400
1946 1074 2021 1474
5---5
filename:93814453111
93814453_S1_C1_P1.jpg
['93814453' '938144531

filename:93833212221
93833212_S2_C2_P1.jpg
['93833212' '93833212221' '1' '2' '2' '1' '局部开裂' '2862' '849' '0' '0'
 '548' '1107' '0' '0' '1']
2862 849 548 1107
2862 849 3410 1956
100---182
['93833212' '93833212221' '2' '2' '2' '1' '局部开裂' '1517' '2095' '0' '0'
 '1306' '776' '0' '0' '1']
1517 2095 1306 776
1517 2095 2823 2871
100---183
['93833212' '93833212221' '3' '2' '2' '1' '局部开裂' '279' '2615' '0' '0'
 '1113' '304' '0' '0' '1']
279 2615 1113 304
279 2615 1392 2919
100---184
filename:93833197212
93833197_S2_C1_P2.jpg
['93833197' '93833197212' '0' '2' '1' '2' '局部开裂' '3852' '4523' '0' '0'
 '208' '476' '0' '0' '1']
3852 4523 208 476
3852 4523 4060 4999
101---185
filename:93833197221
93833197_S2_C2_P1.jpg
['93833197' '93833197221' '1' '2' '2' '1' '局部开裂' '1057' '2279' '0' '0'
 '1531' '640' '0' '0' '1']
1057 2279 1531 640
1057 2279 2588 2919
102---186
['93833197' '93833197221' '2' '2' '2' '1' '局部开裂' '2841' '1310' '0' '0'
 '446' '667' '0' '0' '1']
2841 1310 446 667
2841 1310 3287 1977
102---187

filename:93136907221
93136907_S2_C2_P1.jpg
['93136907' '93136907221' '2' '2' '2' '1' '局部开裂' '2583' '3000' '0' '0'
 '333' '290' '0' '0' '1']
2583 3000 333 290
2583 3000 2916 3290
169---430
['93136907' '93136907221' '0' '2' '2' '1' '局部开裂' '449' '3485' '0' '0'
 '1299' '195' '0' '0' '1']
449 3485 1299 195
449 3485 1748 3680
169---431
['93136907' '93136907221' '1' '2' '2' '1' '局部开裂' '2135' '3270' '0' '0'
 '370' '225' '0' '0' '1']
2135 3270 370 225
2135 3270 2505 3495
169---432
['93136907' '93136907221' '3' '2' '2' '1' '局部开裂' '401' '3035' '0' '0'
 '2461' '689' '0' '0' '1']
401 3035 2461 689
401 3035 2862 3724
169---433
filename:93136362211
93136362_S2_C1_P1.jpg
['93136362' '93136362211' '0' '2' '1' '1' '局部开裂' '4048' '2550' '0' '0'
 '392' '433' '0' '0' '1']
4048 2550 392 433
4048 2550 4440 2983
170---434
['93136362' '93136362211' '1' '2' '1' '1' '局部开裂' '4085' '2605' '0' '0'
 '312' '309' '0' '0' '1']
4085 2605 312 309
4085 2605 4397 2914
170---435
filename:93136362212
93136362_S2_C1_P2.jpg
['9

filename:93822536221
93822536_S2_C2_P1.jpg
['93822536' '93822536221' '2' '2' '2' '1' '局部开裂' '2341' '2357' '0' '0'
 '616' '570' '0' '0' '1']
2341 2357 616 570
2341 2357 2957 2927
240---675
['93822536' '93822536221' '1' '2' '2' '1' '局部开裂' '939' '2853' '0' '0'
 '1496' '435' '0' '0' '1']
939 2853 1496 435
939 2853 2435 3288
240---676
['93822536' '93822536221' '3' '2' '2' '1' '局部开裂' '960' '2317' '0' '0'
 '1961' '974' '0' '0' '1']
960 2317 1961 974
960 2317 2921 3291
240---677
filename:93822532211
93822532_S2_C1_P1.jpg
['93822532' '93822532211' '3' '2' '1' '1' '局部开裂' '4175' '3513' '0' '0'
 '192' '139' '0' '0' '1']
4175 3513 192 139
4175 3513 4367 3652
241---678
['93822532' '93822532211' '2' '2' '1' '1' '局部开裂' '4594' '3629' '0' '0'
 '451' '414' '0' '0' '1']
4594 3629 451 414
4594 3629 5045 4043
241---679
['93822532' '93822532211' '1' '2' '1' '1' '局部开裂' '3178' '2509' '0' '0'
 '183' '213' '0' '0' '1']
3178 2509 183 213
3178 2509 3361 2722
241---680
['93822532' '93822532211' '4' '2' '1' '1' '局部开

filename:93133249122
93133249_S1_C2_P2.jpg
['93133249' '93133249122' '0' '1' '2' '2' '边损' '4604' '3162' '0' '0' '84'
 '17' '0' '0' '1']
4604 3162 84 17
4604 3162 4688 3179
353---875
filename:93133249121
93133249_S1_C2_P1.jpg
['93133249' '93133249121' '0' '1' '2' '1' '边损' '3511' '2587' '0' '0' '58'
 '16' '0' '0' '1']
3511 2587 58 16
3511 2587 3569 2603
354---876
filename:93126035112
93126035_S1_C1_P2.jpg
['93126035' '93126035112' '0' '1' '1' '2' '边损' '1349' '2314' '0' '0' '32'
 '27' '0' '0' '1']
1349 2314 32 27
1349 2314 1381 2341
355---877
filename:93126035121
93126035_S1_C2_P1.jpg
['93126035' '93126035121' '0' '1' '2' '1' '边损' '2401' '2239' '0' '0' '364'
 '261' '0' '0' '1']
2401 2239 364 261
2401 2239 2765 2500
356---878
filename:93124887111
93124887_S1_C1_P1.jpg
['93124887' '93124887111' '0' '1' '1' '1' '边损' '799' '2526' '0' '0' '28'
 '27' '0' '0' '1']
799 2526 28 27
799 2526 827 2553
357---879
filename:93124887112
93124887_S1_C1_P2.jpg
['93124887' '93124887112' '0' '1' '1' '2' '边损' 